In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Embedding, MultiHeadAttention, Dense, LSTM, Input

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

2024-03-24 14:13:57.807821: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 14:13:58.038146: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 14:13:58.038296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 14:13:58.079399: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 14:13:58.166794: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 14:13:58.168165: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
df = pd.read_csv('UpdatedResumeDataSet.csv')
df.head()
df.info()
max_len = 3000
resumes = df['Resume']
categories = df['Category']
Tokenizer = Tokenizer()
Tokenizer.fit_on_texts(resumes)
sequences = Tokenizer.texts_to_sequences(df['Resume'].values)

X = pad_sequences(sequences, maxlen=max_len, padding='post')
y = pd.get_dummies(df['Category']).values

label_encoder = LabelEncoder()
categories_encoded = label_encoder.fit_transform(categories)
categories_encoded = to_categorical(categories_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  96 non-null     object
 1   Resume    96 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


In [4]:
embeddings_dim = 100
embeddings_index = {}
with open('glove.6B.100d.txt', encoding = 'utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(Tokenizer.word_index) + 1, embeddings_dim))
for word, i in Tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
input_layer = Input(shape=(max_len,))

embedding_layer = Embedding(len(Tokenizer.word_index) + 1, embeddings_dim, weights=[embedding_matrix], trainable=False)(input_layer)
lstm_layer = LSTM(units=128, return_sequences=True)(embedding_layer)
attention_layer = MultiHeadAttention(num_heads=2, key_dim=2)(lstm_layer, lstm_layer)
lstm_layer_2 = LSTM(units=64)(attention_layer)

output_layer = Dense(7, activation='softmax')(lstm_layer_2)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=2097152, callbacks=[EarlyStopping(patience=2)])
loss, auc = model.evaluate(X_test, y_test)
print(f"Loss: {loss}")

new_resume = """
KUSHL ALVE  Work Experience :  ●   Web Developer Intern (at Prodigy InfoTech, Mumbai) :   Nov,2023 - Dec,2023  Developed following web applications using HTML, CSS and JS :  1.   Responsive Landing Page.  2.   Stopwatch web application.  3.   Tic-Tac-Toe Application.  4.   Personal Portfolio Website.  Education :  ●   BE Computer Engineering (2022-2026) :  Vivekanand Education Society’s Institute of Technology, Chembur.  Technical Skills :  ●   React.js , Node.js, Express.js  ●   HTML, CSS, JavaScript  ●   PostgreSQL  ●   Python  Projects :  ●   Intelligent Scholarship Portal :  →   used React.js, Node.js, Express.js and PostgreSQL.  → This project aims at providing all scholarships including private ones in the  existing National Scholarship Portal to help save students’ time.  ●   Movie Rating System :  → used Java.  → This project collected various users’ ratings for movies.  Certifications and Achievements :  ●   The Complete Full Stack Web Development Bootcamp 2024 course - Angela Yu  ●   Syrus Hackathon 23’  ●   Received LOR(Letter of Recommendation) from Prodigy InfoTech.  ●   Google Cloud Computing Foundations Skill Badge.  Contact :  ●   +91 9967457425 (mobile)  ●   https://www.linkedin.com/in/kushl-alve-2627541b0/ (LinkedIn)  ●   https://github.com/moodyspider266 (GitHub)  ●   kushlalve007@gmail.com / 2022.kushl.alve@ves.ac.in (Email) 
"""
new_resume = Tokenizer.texts_to_sequences([new_resume])
new_resume = pad_sequences(new_resume, maxlen=max_len)
prediction = model.predict(new_resume)
print(f"Predicted category: {categories[np.argmax(prediction)]}")

Epoch 1/10


2024-03-24 14:14:19.441220: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5472000000 exceeds 10% of free system memory.
2024-03-24 14:14:26.755172: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5472000000 exceeds 10% of free system memory.
